### LICENSE PLATE OBJECT DETECTION

Based on https://huggingface.co/datasets/keremberke/license-plate-object-detection

In [4]:
# Install dependencies and libreries
!pip install torchmetrics
!pip install --upgrade --no-cache-dir gdown
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate migrate.exe run script.py
troubleshoot troubleshoot.exe trust-script.py trust.exe

Jupyter command `jupyter-nbextension` not found.


In [61]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

import torch
import torchvision
import torchsummary
import torchmetrics
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoFeatureExtractor

ModuleNotFoundError: No module named 'transformers'

In [34]:
from datasets import load_dataset, Image

data_set = load_dataset("keremberke/license-plate-object-detection", name="full", split='train', streaming=True)

In [57]:
# Dataset features
data_set.features

{'image_id': Value(dtype='int64', id=None),
 'image': Image(decode=True, id=None),
 'width': Value(dtype='int32', id=None),
 'height': Value(dtype='int32', id=None),
 'objects': Sequence(feature={'id': Value(dtype='int64', id=None), 'area': Value(dtype='int64', id=None), 'bbox': Sequence(feature=Value(dtype='float32', id=None), length=4, id=None), 'category': ClassLabel(names=['license_plate'], id=None)}, length=-1, id=None)}